## Introducere

Această lecție va acoperi:
- Ce este apelarea de funcții și în ce situații se folosește
- Cum să creezi un apel de funcție folosind Azure OpenAI
- Cum să integrezi un apel de funcție într-o aplicație

## Obiective de învățare

După ce vei parcurge această lecție, vei ști și vei înțelege:

- Scopul utilizării apelării de funcții
- Cum să configurezi apelarea de funcții folosind serviciul Azure Open AI
- Cum să proiectezi apeluri de funcții eficiente pentru cazul tău de utilizare


## Înțelegerea apelurilor de funcții

Pentru această lecție, vrem să dezvoltăm o funcționalitate pentru startup-ul nostru educațional care să permită utilizatorilor să folosească un chatbot pentru a găsi cursuri tehnice. Vom recomanda cursuri care se potrivesc nivelului lor de abilități, rolului actual și tehnologiei de interes.

Pentru a realiza acest lucru, vom folosi o combinație de:
 - `Azure Open AI` pentru a crea o experiență de chat pentru utilizator
 - `Microsoft Learn Catalog API` pentru a ajuta utilizatorii să găsească cursuri pe baza cererii lor
 - `Function Calling` pentru a prelua întrebarea utilizatorului și a o trimite către o funcție care va face cererea către API.

Pentru început, să vedem de ce am vrea să folosim apelarea funcțiilor:

print("Mesaje în următoarea cerere:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # obține un nou răspuns de la GPT unde poate vedea răspunsul funcției


print(second_response.choices[0].message)


### De ce să folosești Function Calling

Dacă ai parcurs deja alte lecții din acest curs, probabil că ai înțeles cât de puternice sunt modelele de limbaj de tip Large Language Models (LLMs). Sperăm că ai observat și unele dintre limitele lor.

Function Calling este o funcționalitate din Azure Open AI Service care ajută la depășirea următoarelor limitări:
1) Formatul constant al răspunsurilor
2) Posibilitatea de a folosi date din alte surse ale unei aplicații într-un context de chat

Înainte de function calling, răspunsurile generate de un LLM erau neorganizate și nu aveau un format consecvent. Dezvoltatorii trebuiau să scrie cod complex de validare pentru a se asigura că pot gestiona fiecare variantă de răspuns.

Utilizatorii nu puteau primi răspunsuri de tipul „Care este vremea actuală în Stockholm?”. Asta pentru că modelele erau limitate la perioada în care au fost antrenate cu date.

Hai să analizăm exemplul de mai jos care ilustrează această problemă:

Să presupunem că vrem să creăm o bază de date cu informații despre studenți, ca să le putem recomanda cursul potrivit. Mai jos avem două descrieri de studenți care sunt foarte asemănătoare ca date conținute.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Vrem să trimitem acest lucru către un LLM pentru a analiza datele. Acest lucru poate fi folosit ulterior în aplicația noastră pentru a trimite datele către o API sau pentru a le stoca într-o bază de date.

Hai să creăm două prompturi identice prin care instruim LLM-ul ce informații ne interesează:


Vrem să trimitem acest lucru către un LLM pentru a analiza părțile care sunt importante pentru produsul nostru. Astfel, putem crea două prompturi identice pentru a instrui LLM-ul:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


După ce creăm aceste două prompturi, le vom trimite către LLM folosind `openai.ChatCompletion`. Stocăm promptul în variabila `messages` și atribuim rolul `user`. Acest lucru este pentru a imita un mesaj de la un utilizator scris către un chatbot.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Chiar dacă instrucțiunile sunt aceleași și descrierile sunt similare, putem obține formate diferite pentru proprietatea `Grades`.

Dacă rulezi celula de mai sus de mai multe ori, formatul poate fi `3.7` sau `3.7 GPA`.

Acest lucru se întâmplă deoarece LLM-ul primește date nestructurate sub forma instrucțiunii scrise și returnează tot date nestructurate. Avem nevoie de un format structurat ca să știm la ce să ne așteptăm atunci când stocăm sau folosim aceste date.

Folosind apelarea funcțională, ne putem asigura că primim date structurate înapoi. Când folosim apelarea funcțională, LLM-ul nu apelează sau rulează efectiv nicio funcție. În schimb, creăm o structură pe care LLM-ul să o urmeze pentru răspunsurile sale. Apoi folosim aceste răspunsuri structurate ca să știm ce funcție să rulăm în aplicațiile noastre.


![Diagrama fluxului de apelare a funcțiilor](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.ro.png)


### Exemple de utilizare pentru apelarea funcțiilor

**Apelarea unor instrumente externe**  
Chatboții sunt foarte buni la a oferi răspunsuri la întrebările utilizatorilor. Prin folosirea apelării de funcții, chatboții pot folosi mesajele utilizatorilor pentru a îndeplini anumite sarcini. De exemplu, un student poate cere chatbotului: „Trimite un email profesorului meu în care să-i spun că am nevoie de mai mult ajutor la această materie”. Acest lucru poate declanșa un apel către funcția `send_email(to: string, body: string)`

**Crearea de interogări API sau către baze de date**  
Utilizatorii pot găsi informații folosind limbaj natural, care este apoi transformat într-o interogare formatată sau o cerere API. Un exemplu ar fi un profesor care întreabă „Cine sunt studenții care au finalizat ultima temă”, ceea ce ar putea apela o funcție numită `get_completed(student_name: string, assignment: int, current_status: string)`

**Crearea de date structurate**  
Utilizatorii pot lua un bloc de text sau un fișier CSV și pot folosi LLM-ul pentru a extrage informațiile importante din acesta. De exemplu, un student poate transforma un articol de pe Wikipedia despre acorduri de pace pentru a crea fișe de învățare AI. Acest lucru se poate face folosind o funcție numită `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Crearea primei tale apelări de funcție

Procesul de creare a unei apelări de funcție include 3 pași principali:
1. Apelezi API-ul Chat Completions cu o listă de funcții și un mesaj de la utilizator
2. Citești răspunsul modelului pentru a efectua o acțiune, adică să execuți o funcție sau un apel API
3. Faci o altă apelare către API-ul Chat Completions cu răspunsul de la funcția ta pentru a folosi acea informație la crearea unui răspuns pentru utilizator.


![Fluxul unui apel de funcție](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.ro.png)


### Elemente ale unui apel de funcție

#### Introducerea utilizatorului

Primul pas este să creezi un mesaj de la utilizator. Acesta poate fi atribuit dinamic, preluând valoarea dintr-un câmp de text, sau poți seta o valoare aici. Dacă este prima dată când lucrezi cu API-ul Chat Completions, trebuie să definim `role` și `content` pentru mesaj.

`role` poate fi fie `system` (pentru a stabili reguli), `assistant` (modelul) sau `user` (utilizatorul final). Pentru apelarea funcțiilor, vom seta acest rol ca `user` și vom folosi o întrebare exemplu.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Crearea funcțiilor.

În continuare vom defini o funcție și parametrii acesteia. Vom folosi aici doar o funcție numită `search_courses`, dar poți crea mai multe funcții.

**Important** : Funcțiile sunt incluse în mesajul de sistem către LLM și vor fi luate în calcul la numărul de tokenuri disponibile.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definiții**

`name` - Numele funcției pe care dorim să o apelăm.

`description` - Aceasta este descrierea modului în care funcționează funcția. Aici este important să fii specific și clar.

`parameters` - O listă de valori și formatul pe care vrei ca modelul să le genereze în răspunsul său.


`type` - Tipul de date în care vor fi stocate proprietățile.

`properties` - Lista valorilor specifice pe care modelul le va folosi pentru răspunsul său.


`name` - numele proprietății pe care modelul îl va folosi în răspunsul său formatat

`type` - Tipul de date al acestei proprietăți

`description` - Descrierea proprietății specifice


**Opțional**

`required` - proprietate necesară pentru ca apelul funcției să fie finalizat


### Efectuarea apelului funcției
După ce am definit o funcție, trebuie acum să o includem în apelul către Chat Completion API. Facem acest lucru adăugând `functions` la cerere. În acest caz, `functions=functions`.

Există și opțiunea de a seta `function_call` la `auto`. Asta înseamnă că vom lăsa LLM-ul să decidă ce funcție ar trebui apelată, pe baza mesajului utilizatorului, în loc să o atribuim noi.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Acum să analizăm răspunsul și să vedem cum este formatat:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Poți observa că numele funcției este apelat și, din mesajul utilizatorului, LLM-ul a reușit să găsească datele potrivite pentru argumentele funcției.


## 3. Integrarea apelurilor de funcții într-o aplicație.

După ce am testat răspunsul formatat de la LLM, acum putem integra acest lucru într-o aplicație.

### Gestionarea fluxului

Pentru a integra acest lucru în aplicația noastră, să urmăm pașii de mai jos:

Mai întâi, să facem apelul către serviciile Open AI și să stocăm mesajul într-o variabilă numită `response_message`.


In [ ]:
response_message = response.choices[0].message

Acum vom defini funcția care va apela API-ul Microsoft Learn pentru a obține o listă de cursuri:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Ca bună practică, vom verifica apoi dacă modelul dorește să apeleze o funcție. După aceea, vom crea una dintre funcțiile disponibile și o vom asocia cu funcția care este apelată. 
Apoi vom prelua argumentele funcției și le vom mapa la argumentele provenite de la LLM.

La final, vom adăuga mesajul de apelare a funcției și valorile returnate de mesajul `search_courses`. Acest lucru oferă LLM-ului toate informațiile de care are nevoie
pentru a răspunde utilizatorului folosind limbaj natural.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Provocare de cod

Felicitări! Pentru a-ți continua învățarea despre Azure Open AI Function Calling, poți construi: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Mai mulți parametri ai funcției care ar putea ajuta cursanții să găsească mai multe cursuri. Poți găsi parametrii disponibili ai API-ului aici:
 - Creează o altă apelare de funcție care să preia mai multe informații de la cursant, cum ar fi limba maternă
 - Implementează gestionarea erorilor atunci când apelul funcției și/sau apelul API nu returnează niciun curs potrivit



---

**Declinarea responsabilității**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși depunem eforturi pentru acuratețe, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original, în limba sa nativă, trebuie considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de oameni. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea în urma utilizării acestei traduceri.
